In [69]:
import pandas as pd
import numpy as np
import nltk
import re

from nltk.corpus import stopwords
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, Dropout, Dense, Flatten, GlobalMaxPooling1D, Embedding, Conv1D, Input
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer, one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [70]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/doutoum/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/doutoum/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/doutoum/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/doutoum/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [71]:
dataset = [
    'This movie is excellent',
    'I loved the movie, it was fantastic',
    'The film is brilliant, you should watch',
    'wonderful movie',
    'one of the best film ever',
    'fantastic film to watch',
    'great movie',
    'Acting and direction is brilliant',
    
    'poor acting',
    'horrible film',
    'pathetic acting',
    'The film is very boring',
    'I wasted my money',
    'I did not like the film',
    'not recommended',
    'It was a poor story'
]

labels = np.array([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0])
labels.shape
len(dataset)

16

In [72]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(dataset)
vocabulary_length = len(tokenizer.word_index) + 1
integer_sentences = tokenizer.texts_to_sequences(dataset)
print(integer_sentences)

[[14, 3, 4, 15], [5, 16, 2, 3, 7, 8, 9], [2, 1, 4, 10, 17, 18, 11], [19, 3], [20, 21, 2, 22, 1, 23], [9, 1, 24, 11], [25, 3], [6, 26, 27, 4, 10], [12, 6], [28, 1], [29, 6], [2, 1, 4, 30, 31], [5, 32, 33, 34], [5, 35, 13, 36, 2, 1], [13, 37], [7, 8, 38, 12, 39]]


In [73]:
from nltk.tokenize import word_tokenize
token_count = lambda sent : len(word_tokenize(sent))
max_sentence = max(dataset, key=token_count)
max_sentence_length = len(word_tokenize(max_sentence))

padded_reviews = pad_sequences(integer_sentences, max_sentence_length, padding='post')
padded_reviews

array([[14,  3,  4, 15,  0,  0,  0,  0],
       [ 5, 16,  2,  3,  7,  8,  9,  0],
       [ 2,  1,  4, 10, 17, 18, 11,  0],
       [19,  3,  0,  0,  0,  0,  0,  0],
       [20, 21,  2, 22,  1, 23,  0,  0],
       [ 9,  1, 24, 11,  0,  0,  0,  0],
       [25,  3,  0,  0,  0,  0,  0,  0],
       [ 6, 26, 27,  4, 10,  0,  0,  0],
       [12,  6,  0,  0,  0,  0,  0,  0],
       [28,  1,  0,  0,  0,  0,  0,  0],
       [29,  6,  0,  0,  0,  0,  0,  0],
       [ 2,  1,  4, 30, 31,  0,  0,  0],
       [ 5, 32, 33, 34,  0,  0,  0,  0],
       [ 5, 35, 13, 36,  2,  1,  0,  0],
       [13, 37,  0,  0,  0,  0,  0,  0],
       [ 7,  8, 38, 12, 39,  0,  0,  0]], dtype=int32)

In [74]:
embedded_dict = dict()
glove_embeddings = open('../data/glove.6B.100d.txt', encoding='utf-8')

In [75]:
for embeddings in glove_embeddings:
    embeddings_tokens = embeddings.split()
    emb_word = embeddings_tokens[0]
    emb_vector = np.asarray(embeddings_tokens[1:], dtype='float32')
    embedded_dict[emb_word] = emb_vector
glove_embeddings.close()

In [76]:
embedd_mat = np.zeros((vocabulary_length, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embedded_dict.get(word)
    if embedding_vector is not None:
        embedd_mat[index] = embedding_vector
embedd_mat.shape

(40, 100)

In [77]:
def enbedded_model():
    embedding_inputs = Input(shape=(max_sentence_length))
    embedding_layer = Embedding(vocabulary_length, 100, weights=[embedd_mat], trainable=False)(embedding_inputs)
    flatten_layer = Flatten()(embedding_layer)
    output_layer = Dense(1, activation='sigmoid')(flatten_layer)
    return Model(inputs=embedding_inputs, outputs=output_layer)

model = enbedded_model()

In [78]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 8)]               0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 8, 100)            4000      
_________________________________________________________________
flatten_4 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 801       
Total params: 4,801
Trainable params: 801
Non-trainable params: 4,000
_________________________________________________________________


In [79]:
print(padded_reviews.shape)
print(embedd_mat.shape)
model.fit(padded_reviews, labels, epochs=100, verbose=1)
loss, accuracy = model.evaluate(padded_reviews, labels, verbose=0)
print('Accuracy: %f'%(accuracy*100))

(16, 8)
(40, 100)
Epoch 1/100
1/1 [==============================] - 1s 509ms/step - loss: 0.7067 - acc: 0.5625
Epoch 2/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6848 - acc: 0.6250
Epoch 3/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6637 - acc: 0.6250
Epoch 4/100
1/1 [==============================] - 0s 4ms/step - loss: 0.6432 - acc: 0.7500
Epoch 5/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6233 - acc: 0.7500
Epoch 6/100
1/1 [==============================] - 0s 3ms/step - loss: 0.6041 - acc: 0.7500
Epoch 7/100
1/1 [==============================] - 0s 3ms/step - loss: 0.5855 - acc: 0.7500
Epoch 8/100
1/1 [==============================] - 0s 4ms/step - loss: 0.5676 - acc: 0.7500
Epoch 9/100
1/1 [==============================] - 0s 3ms/step - loss: 0.5502 - acc: 0.8125
Epoch 10/100
1/1 [==============================] - 0s 3ms/step - loss: 0.5334 - acc: 0.8125
Epoch 11/100
1/1 [==============================] - 0s 3ms/

Epoch 89/100
1/1 [==============================] - 0s 2ms/step - loss: 0.1010 - acc: 1.0000
Epoch 90/100
1/1 [==============================] - 0s 2ms/step - loss: 0.0996 - acc: 1.0000
Epoch 91/100
1/1 [==============================] - 0s 2ms/step - loss: 0.0983 - acc: 1.0000
Epoch 92/100
1/1 [==============================] - 0s 2ms/step - loss: 0.0970 - acc: 1.0000
Epoch 93/100
1/1 [==============================] - 0s 2ms/step - loss: 0.0957 - acc: 1.0000
Epoch 94/100
1/1 [==============================] - 0s 3ms/step - loss: 0.0945 - acc: 1.0000
Epoch 95/100
1/1 [==============================] - 0s 2ms/step - loss: 0.0933 - acc: 1.0000
Epoch 96/100
1/1 [==============================] - 0s 2ms/step - loss: 0.0921 - acc: 1.0000
Epoch 97/100
1/1 [==============================] - 0s 2ms/step - loss: 0.0909 - acc: 1.0000
Epoch 98/100
1/1 [==============================] - 0s 2ms/step - loss: 0.0898 - acc: 1.0000
Epoch 99/100
1/1 [==============================] - 0s 2ms/step - loss